In [1]:
import sys
import os
import json
import glob
import cv2
import tqdm
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ExifTags
import numpy as np

In [2]:
CLASSES = ['numberplate']
STATES  = ["train", "val"]

PATH_TO_JSON    = "./npdata/{}/via_region_data.json"
PATH_TO_RES_ANN = "./npdata/labels/{}"
PATH_TO_IMAGES  = "./npdata/images/{}"

In [3]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)

In [6]:
def load_annotations(path_to_res_ann, path_to_images, path_to_json, classes = ['numberplate'], debug=True):
    with open(path_to_json) as ann:
        annData = json.load(ann)
    cat2label = {k: i for i, k in enumerate(classes)}
    image_list = annData
    
    for _id in tqdm.tqdm(image_list["_via_img_metadata"]):
        image_id = image_list["_via_img_metadata"][_id]["filename"]
        filename = f'{path_to_images}/{image_id}'
        pil_image = Image.open(filename)
        image = np.array(pil_image)
        width, height= pil_image.size
        
        to_txt_data = []
        for region in image_list["_via_img_metadata"][_id]["regions"]:
            label_id  = 0
            #label_id = cat2label[bbox_name]
            if region["shape_attributes"].get("all_points_x", None) is None:
                continue
            if region["shape_attributes"].get("all_points_y", None) is None:
                continue
            bbox = [
                min(region["shape_attributes"]["all_points_x"]),
                min(region["shape_attributes"]["all_points_y"]),
                max(region["shape_attributes"]["all_points_x"]),
                max(region["shape_attributes"]["all_points_y"]),
            ]
            
            w = bbox[2] - bbox[0]
            h = bbox[3] - bbox[1]
            
            mx = bbox[0]+w/2
            my = bbox[1]+h/2
            
            # class x_center y_center width height
            yolo_bbox = " ".join([str(item) for item in [label_id, mx/width, my/height, w/width, h/height]])
            to_txt_data.append(yolo_bbox)
            if debug:
                cv2.rectangle(image, 
                    (int(bbox[0]), int(bbox[1])), 
                    (int(bbox[2]), int(bbox[3])), 
                    (0,120,255), 
                    3)
        
        res_path =  f'{path_to_res_ann}/{".".join(image_id.split(".")[:-1])}.txt'
        if debug:
            print(res_path)
            print("\n".join(to_txt_data))
            print("______________________")
            plt.imshow(image)
            plt.show()
            
        else:
            with open(res_path, "w") as wFile:
                wFile.write("\n".join(to_txt_data))
            pil_image.save(filename)

<img src="https://user-images.githubusercontent.com/26833433/98809572-0bc4d580-241e-11eb-844e-eee756f878c2.png">

In [7]:
for state in STATES:
    path_to_json    = PATH_TO_JSON.format(state)
    path_to_res_ann = PATH_TO_RES_ANN.format(state)
    path_to_images  = PATH_TO_IMAGES.format(state)
    
    load_annotations(path_to_res_ann, path_to_images, path_to_json, debug=False)

100%|██████████| 376/376 [00:04<00:00, 80.28it/s]
